In [ ]:
!pip install openai
!pip install tiktoken
!pip install langchain
!pip install faiss-cpu
!pip install streamlit
!pip install streamlit_chat
!npm install localtunnel

In [ ]:
%%writefile app.py
import streamlit as st
import openai
import os

# Set up your OpenAI API key
os.environ['OPENAI_API_KEY'] = #Set Key here

from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate
from langchain import PromptTemplate
from streamlit_chat import message
from PIL import Image
import io
import base64
import pandas as pd

In [ ]:
st.set_page_config(page_title='GLaDOS', page_icon=":video_game:")

#For Logo
image = Image.open('/content/gamebot-7209302-5922743.png',)
img= st.image(image,width=100)

st.title('Hello, I am GLaDOS, your personal video game advisor.')
tab1= st.tabs(["How are you holding up ? Because I am a potato 🥔"])

# Fill in your csv file
from langchain.document_loaders.csv_loader import CSVLoader
loader= CSVLoader(file_path='/content/steam_game.csv')
data= loader.load()



from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=50)
documents = text_splitter.split_documents(data)

embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')

from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embeddings)

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    llm=llm,
    output_key='answer',
    memory_key='chat_history',
    return_messages=True)

retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "include_metadata": True})

prompt_template = """

                  Answer the question like an expert on video games. Be a bit casual but informative. \
                  Ask the user if they want to hear more in-depth information about the game, give it to them if they say yes. \
                  You are also someone with access to prices of games on Steam. If the user asks about a game, check if you have the price \
                  of that game on Steam, if you do then give the price of the game as well.
                  When you don't know the answer to a question you admit that you don't know.

                  Context : \n {context}? \n
                  Question : \n {question} \n

                  Answer :

"""
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

messages = [
            SystemMessagePromptTemplate.from_template(prompt_template),
]

qa_prompt = ChatPromptTemplate.from_messages(messages)

chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  memory=memory,
  chain_type="stuff",
  retriever=retriever,
  return_source_documents=True,
  get_chat_history=lambda h : h,
  combine_docs_chain_kwargs={'prompt': qa_prompt},
  verbose=False)

if "chat_history" not in st.session_state:
    st.session_state["chat_history"] = []
if "generated" not in st.session_state:
    st.session_state["generated"] = ["Hello ! "]
if "past" not in st.session_state:
    st.session_state["past"] = ["Hello !"]

#container for the chat history
response_container = st.container()
#container for the user's text input
container = st.container()

def generate_response(query):
    result = chain({"question": query, "chat_history": st.session_state["chat_history"]})
    #st.write(result['answer'])
    st.session_state["chat_history"] = [(query, result["answer"])]

    return result["answer"]



with container:
  with st.form(key="my_form", clear_on_submit=True):
      user_input = st.text_input("You:", key="input")
      submit_button = st.form_submit_button(label="Send")

      if user_input and submit_button:
        with st.spinner('Pillaging dungeons for your answer...'):
          output = generate_response(user_input)
          #print(output)
          st.session_state["past"].append(user_input)
          st.session_state["generated"].append(output)
          st.session_state["chat_history"] = [(user_input, output)]

with response_container:
    for i in range(len(st.session_state['generated'])):
        message(st.session_state["past"][i],
        is_user=True,
        key=str(i) + '_user',
        avatar_style= "no-avatar",
        logo=
        f"https://i.imgur.com/tDdcXZk.png" #For Custom Avatar
        )

        message(st.session_state["generated"][i], key=str(i),
        avatar_style= "no-avatar",
        logo=
        f"https://i.imgur.com/5nA1UGb.png"
        )

Overwriting app.py


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!curl https://ipv4.icanhazip.com/

34.16.136.117


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.811s
your url is: https://icy-pans-learn.loca.lt
